In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [86]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=5)

In [87]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [88]:
torch.set_grad_enabled(False)

the pytorch gradient calculation is being turned off as it is not required until the training begins, no loss is being generated yet

In [89]:
network = Network()

In [90]:
sample = next(iter(train_set))

In [91]:
image, label = sample

In [92]:
image.shape

torch.Size([1, 28, 28])

a single colour channel image which is 28x28

the network requires an input of a batch, therefore we can generate a batch with only one image by reshaping,

the batch tensor is as follows: (batchsize, in_channels, height, width)

In [93]:
pred = network(image.unsqueeze(0))

In [94]:
pred

tensor([[ 0.1406, -0.0330,  0.0685,  0.0430,  0.0562,  0.0629,  0.0999, -0.0022, -0.0102, -0.1140]])

In [95]:
pred.shape

torch.Size([1, 10])

In [96]:
F.softmax(pred, dim=1)

tensor([[0.1113, 0.0936, 0.1036, 0.1010, 0.1023, 0.1030, 0.1069, 0.0965, 0.0957, 0.0863]])

the shape of the prediction tensor shows that there is one image, with 10 different predictions

using the softmax function , we can see these are probabilities

In [124]:
pred.argmax(dim=1)

tensor([0, 0, 0, 0, 0])

In [98]:
label

9

the prediction shows that the highest label prediction is an 8, the real label value was 9, which is wrong

In [99]:
batch = next(iter(train_loader))

In [100]:
images, labels = batch

In [101]:
images.shape

torch.Size([5, 1, 28, 28])

In [102]:
pred = network(images)

In [103]:
pred.shape

torch.Size([5, 10])

In [104]:
pred

tensor([[ 0.1406, -0.0330,  0.0685,  0.0430,  0.0562,  0.0629,  0.0999, -0.0022, -0.0102, -0.1140],
        [ 0.1432, -0.0339,  0.0629,  0.0431,  0.0542,  0.0635,  0.0948, -0.0105, -0.0148, -0.1067],
        [ 0.1395, -0.0348,  0.0659,  0.0450,  0.0545,  0.0609,  0.0955, -0.0117, -0.0114, -0.1121],
        [ 0.1455, -0.0358,  0.0659,  0.0447,  0.0553,  0.0595,  0.0925, -0.0103, -0.0135, -0.1116],
        [ 0.1441, -0.0392,  0.0654,  0.0466,  0.0508,  0.0581,  0.0991, -0.0089, -0.0132, -0.1135]])

In [125]:
F.softmax(pred, dim=1)

tensor([[0.1113, 0.0936, 0.1036, 0.1010, 0.1023, 0.1030, 0.1069, 0.0965, 0.0957, 0.0863],
        [0.1118, 0.0936, 0.1032, 0.1011, 0.1023, 0.1032, 0.1065, 0.0959, 0.0954, 0.0871],
        [0.1114, 0.0936, 0.1035, 0.1014, 0.1023, 0.1030, 0.1066, 0.0958, 0.0958, 0.0866],
        [0.1121, 0.0935, 0.1035, 0.1013, 0.1024, 0.1028, 0.1063, 0.0959, 0.0956, 0.0867],
        [0.1119, 0.0932, 0.1035, 0.1015, 0.1020, 0.1027, 0.1070, 0.0961, 0.0956, 0.0865]])

In [122]:
pred.argmax(dim=1)

tensor([0, 0, 0, 0, 0])

In [106]:
labels

tensor([9, 0, 0, 3, 0])

using this batch of 5 images, we can see that two were guessed correctly